In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc
import os
print(os.getcwd())


C:\Users\tianh\Dropbox\2017\kaggle\zillow\output


In [2]:
train = pd.read_csv('../input/train_2016_v2.csv')


In [3]:
prop = pd.read_csv('../input/properties_2016.csv')

c:\toolkits\anaconda2-4.2.0\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
for c, dtype in zip(prop.columns, prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)

In [5]:
df_train = train.merge(prop, how='left', on='parcelid')


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90275 entries, 0 to 90274
Data columns (total 60 columns):
parcelid                        90275 non-null int64
logerror                        90275 non-null float64
transactiondate                 90275 non-null object
airconditioningtypeid           28781 non-null float32
architecturalstyletypeid        261 non-null float32
basementsqft                    43 non-null float32
bathroomcnt                     90275 non-null float32
bedroomcnt                      90275 non-null float32
buildingclasstypeid             16 non-null float32
buildingqualitytypeid           57364 non-null float32
calculatedbathnbr               89093 non-null float32
decktypeid                      658 non-null float32
finishedfloor1squarefeet        6856 non-null float32
calculatedfinishedsquarefeet    89614 non-null float32
finishedsquarefeet12            85596 non-null float32
finishedsquarefeet13            33 non-null float32
finishedsquarefeet15        

In [7]:
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)


((90275, 55), (90275L,))


In [8]:
for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

In [9]:
del df_train; gc.collect()

14

In [10]:
split = 90000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
x_valid = x_valid.values.astype(np.float32, copy=False)

In [11]:
d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)


In [12]:

params = {}
params['learning_rate'] = 0.002
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['sub_feature'] = 0.5
params['num_leaves'] = 60
params['min_data'] = 500
params['min_hessian'] = 1

In [13]:
watchlist = [d_valid]
clf = lgb.train(params, d_train, 2000, watchlist)

del d_train, d_valid; gc.collect()
del x_train, x_valid; gc.collect()

[1]	valid_0's l1: 0.072129
[2]	valid_0's l1: 0.0721315
[3]	valid_0's l1: 0.0721335
[4]	valid_0's l1: 0.0721333
[5]	valid_0's l1: 0.0721336
[6]	valid_0's l1: 0.0721307
[7]	valid_0's l1: 0.072128
[8]	valid_0's l1: 0.0721279
[9]	valid_0's l1: 0.0721264
[10]	valid_0's l1: 0.0721238
[11]	valid_0's l1: 0.0721229
[12]	valid_0's l1: 0.0721214
[13]	valid_0's l1: 0.0721213
[14]	valid_0's l1: 0.072122
[15]	valid_0's l1: 0.0721201
[16]	valid_0's l1: 0.0721216
[17]	valid_0's l1: 0.0721207
[18]	valid_0's l1: 0.072117
[19]	valid_0's l1: 0.0721199
[20]	valid_0's l1: 0.0721196
[21]	valid_0's l1: 0.072116
[22]	valid_0's l1: 0.0721133
[23]	valid_0's l1: 0.0721157
[24]	valid_0's l1: 0.0721171
[25]	valid_0's l1: 0.0721135
[26]	valid_0's l1: 0.0721156
[27]	valid_0's l1: 0.072113
[28]	valid_0's l1: 0.0721129
[29]	valid_0's l1: 0.0721092
[30]	valid_0's l1: 0.0721071
[31]	valid_0's l1: 0.0721036
[32]	valid_0's l1: 0.0721043
[33]	valid_0's l1: 0.0721024
[34]	valid_0's l1: 0.0721013
[35]	valid_0's l1: 0.0721035


0